## Reimplemenation of Mask R-CNN on Cowbird Dataset

In [1]:
import torchvision
import os
import argparse
import numpy as np
import cv2
import matplotlib.pyplot as plt
import copy
import time
import _init_paths
import torch
from torchvision import transforms as T
from torchvision.models.detection.transform import GeneralizedRCNNTransform
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
from torchvision.models.detection.mask_rcnn import MaskRCNN
from pycocotools.coco import COCO
from datasets import MaskRCNN_Dataset


from pycocotools.mask import decode
from utils.img_utils import dialate_boxes
import random
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torch.utils.data
import torchvision.models.segmentation


In [2]:
torch.cuda.empty_cache()

In [3]:
parser = argparse.ArgumentParser()
parser.add_argument('--root', default='data/cowbird/images', help='Path to image folder')
parser.add_argument('--annfile', default='data/cowbird/annotations/instance_train.json', help='Path to annotation')
parser.add_argument('--index', type=int, default=8, help='Index to the dataset for an example')
parser.add_argument('--outdir', type=str, default='tests_detection', help='Folder for output images')

_StoreAction(option_strings=['--outdir'], dest='outdir', nargs=None, const=None, default='tests_detection', type=<class 'str'>, choices=None, help='Folder for output images', metavar=None)

In [4]:
batchSize=2
imageSize=[256,256]
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# args = parser.parse_args()
root = '/mnt/d/Research/avian-mesh/data/cowbird/images'
annfile = '/mnt/d/Research/avian-mesh/data/cowbird/annotations/instance_train.json'

#### We will use imported MaskRCNN_Dataset
#### but we will define our own loadData() function!

In [8]:
def loadData(dataset, batchSize=3):
  batch_Imgs=[]
  batch_Data=[]
  for i in range(batchSize):
        idx = random.randint(0, len(dataset))
        img, data = dataset[idx]

        batch_Imgs.append(img)
        batch_Data.append(data)  
  
  batch_Imgs=torch.stack([torch.as_tensor(d) for d in batch_Imgs],0)
  batch_Imgs = batch_Imgs.swapaxes(1, 3).swapaxes(2, 3)
  return batch_Imgs, batch_Data

### Load Dataset

In [5]:
normalize_out = T.Compose([
            T.ToTensor(),
            T.Normalize(mean=[0.406, 0.456, 0.485], std=[0.225, 0.224, 0.229])
        ])
# Split data into train and validation, and create dataloaders for them
dataset = MaskRCNN_Dataset(root=root, annfile=annfile, scale_factor=0.25, transform=normalize_out, output_size=128)
train_data, val_data = torch.utils.data.random_split(dataset, [len(list(dataset))-100, 100])

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


/mnt/d/Research/avian-mesh/my_tests_maskrcnn/../datasets/cowbird_dataset.py:197: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:210.)
  data['bboxes'] = torch.tensor(data['bboxes'])


In [ ]:
# imgs, targets = loadData(train_data)
# print(targets)

[{'boxes': tensor([[1234., 1041., 1324., 1131.],
        [1383., 1042., 1439., 1099.],
        [1524.,  347., 1724.,  547.],
        [1352.,  304., 1454.,  407.],
        [ 917.,  276., 1035.,  394.],
        [ 763.,  277.,  883.,  397.],
        [ 528.,  288.,  731.,  492.],
        [ 611.,  255.,  684.,  328.]]), 'labels': tensor([1, 1, 1, 1, 1, 1, 1, 1]), 'masks': tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  .

### Load Pretrained Mask R-CNN

In [6]:
# Load a maskRCNN finetuned on our birds
network_transform = GeneralizedRCNNTransform(800, 1333, (0,0,0), (1,1,1))
backbone = resnet_fpn_backbone(backbone_name='resnet101', pretrained=False)
model = MaskRCNN(backbone, num_classes=2)
model.transform = network_transform
model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
Adam_opt = torch.optim.Adam(params)
model.train()

MaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=(0, 0, 0), std=(1, 1, 1))
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=1e-05)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=1e-05)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=1e-05)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=1e-05)
          (relu): ReLU(inplace=True)
    

### Training for 15 epochs

In [9]:
for i in range(15):
   images, targets = loadData(train_data, batchSize=2)
   # print(targets[0])
   images = list(image.to(device) for image in images)
   targets=[{k: v.to(device) for k,v in t.items()} for t in targets]
   
   if i == 0:
        print(targets[0])
   
   Adam_opt.zero_grad()
   loss_dict = model(images, targets)
   losses = sum(loss for loss in loss_dict.values())
   
   losses.backward()
   Adam_opt.step()
   
   print(i,'loss:', losses.item())
   if i%5==0:
           torch.save(model.state_dict(), str(i)+".torch")
           print("Save model to:",str(i)+".torch")

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.